## 1. 뉴스제목 가져오기
* user-agent 요청헤더를 반드시 설정해야 한다.

In [1]:
!pip show requests

Name: requests
Version: 2.32.2
Summary: Python HTTP for Humans.
Home-page: https://requests.readthedocs.io
Author: Kenneth Reitz
Author-email: me@kennethreitz.org
License: Apache-2.0
Location: /opt/anaconda3/lib/python3.12/site-packages
Requires: certifi, charset-normalizer, idna, urllib3
Required-by: anaconda-catalogs, anaconda-client, anaconda-cloud-auth, anaconda-project, conda, conda-build, conda-repo-cli, conda_package_streaming, cookiecutter, datashader, intake, jupyterlab_server, panel, requests-file, requests-toolbelt, Sphinx, streamlit, tldextract


In [2]:
!pip show beautifulsoup4

Name: beautifulsoup4
Version: 4.12.3
Summary: Screen-scraping library
Home-page: 
Author: 
Author-email: Leonard Richardson <leonardr@segfault.org>
License: MIT License
Location: /opt/anaconda3/lib/python3.12/site-packages
Requires: soupsieve
Required-by: conda-build, nbconvert


In [4]:
import requests
import bs4
from bs4 import BeautifulSoup

In [5]:
print(f'request ver {requests.__version__}' )
print('beautifulsoup {}'.format(bs4.__version__))

request ver 2.32.2
beautifulsoup 4.12.3


### 1. 뉴스 제목 추출하기

In [31]:
# IT/과학 뉴스
req_param = {
    'sid' : 105
}
url = 'https://news.naver.com/section/{sid}'.format(**req_param)
print(url)

# 요청 헤더 설정 : 브라우저 정보
req_header = {
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/104.0.0.0 Safari/537.36'
}

# requests 모듈을 사용해서 http 통신
res = requests.get(url, headers=req_header)
print(type(res))
print(res.status_code)
print(res.ok)
if res.ok:
    # 소스보기 텍스트 가져오기
    html = res.text
    # parsing을 하귀 위해서 BeautifulSoup 객체생성
    soup = BeautifulSoup(html, "html.parser")
    # print(type(soup))
    '''
        <div class='sa_text'>
            <a href='https://n.news.naver.com/mnews/article/092/030029302' class='sa_text_title'>
                <strong class="sa_text_strong">엔씨소프트, 2분기 영업익 88억...실적 개선 박차</strong>
    '''
    # print(len(soup.select("div.sa_text a[href*='https://n.news.naver.com/mnews/article']"))) 88개
    # print(len(soup.select("div.sa_text a.sa_text_title"))) 44개
    atag_list = soup.select("div.sa_text a.sa_text_title")
    # print(type(atag_list))
    for atag in atag_list:
        # text속성 : tag사이의 content, strip() : 공백제거
        title = atag.text.strip()
        # href 속성의 값을 추출하기
        url = atag['href']
        print(f'news 제목 = {title}')
        print(f'링크 = {url}')
        print('----------')

else:
    print(f'Error 발생 {res.status_code}')

https://news.naver.com/section/105
<class 'requests.models.Response'>
200
True
news 제목 = 데브시스터즈, 2분기 영업익 49억원…"연내 '쿠키런' 인도 진출로 매출 확대"
링크 = https://n.news.naver.com/mnews/article/003/0012712051
----------
news 제목 = 우주항공청 하반기 채용 시작…'찾아가는 채용설명회' 개최
링크 = https://n.news.naver.com/mnews/article/008/0005073553
----------
news 제목 = 낯선 이 작물이 파킨슨병 효과 `수퍼푸드`?...국산 신품종 1호  나왔다
링크 = https://n.news.naver.com/mnews/article/029/0002893041
----------
news 제목 = SKT, 2분기 영업익 5375억원…전년비 16%↑
링크 = https://n.news.naver.com/mnews/article/215/0001174047
----------
news 제목 = '7777'·'1004' 잡아라…KT, 선호번호 1만 개 추첨
링크 = https://n.news.naver.com/mnews/article/055/0001179036
----------
news 제목 = 스타벅스-카카오, 대학생 전용 혜택 프로그램 선보인다
링크 = https://n.news.naver.com/mnews/article/029/0002893052
----------
news 제목 = 신유빈·안세영·김우민 등 '팀 갤럭시' 선수들 삼성 올림픽 체험관 방문
링크 = https://n.news.naver.com/mnews/article/366/0001010068
----------
news 제목 = NHN, 2분기 영업이익 285억원…전년比 36.3%↑
링크 = https://n.news.naver.com/mnews/article/008/0005073406
--------

### 1.1 뉴스제목 추출하는 함수 선언하기

In [36]:
import requests
from bs4 import BeautifulSoup

def print_news(sid, section):    
    # 요청 Parameter
    req_param = {
    'sid': sid
    }
    url = 'https://news.naver.com/section/{sid}'.format(**req_param)
    
    print(f'======> {url} {section} 뉴스 <======')
    
    # 요청 헤더 설정 : 브라우저 정보
    req_header = {
        'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/104.0.0.0 Safari/537.36'
    }

    # requests 모듈을 사용해서 http 통신
    res = requests.get(url, headers=req_header)
    if res.ok:
        # 소스보기 텍스트 가져오기
        html = res.text
        # parsing을 하귀 위해서 BeautifulSoup 객체생성
        soup = BeautifulSoup(html, "html.parser")
        atag_list = soup.select("div.sa_text a.sa_text_title")
        for atag in atag_list:
            # text속성 : tag사이의 content, strip() : 공백제거
            title = atag.text.strip()
            # href 속성의 값을 추출하기
            url = atag['href']
            print(f'news 제목 = {title}')
            print(f'링크 = {url}')
            print('----------')
    
    else:
        print(f'Error 발생 {res.status_code}')

In [37]:
section_dict = {100:'정치',101:'경제',102:'사회',103:'생활/문화',104:'세계',105:'IT/과학'}
print_news(103,section_dict[103])

======> https://news.naver.com/section/103 생활/문화 뉴스 <======
news 제목 = 공공도서관 이용자 지속 증가…"수도권·광역시 도서관 더 필요"
링크 = https://n.news.naver.com/mnews/article/421/0007712567
----------
news 제목 = 메르세데스 AMG가 인정한 성능…GT 쿠페 신차에 한국타이어 장착
링크 = https://n.news.naver.com/mnews/article/421/0007712505
----------
news 제목 = 전국 곳곳 소나기…체감온도 35도 안팎 무더위 계속[930 날씨]
링크 = https://n.news.naver.com/mnews/article/056/0011775769
----------
news 제목 = 폭스바겐코리아, 프리미엄 SUV '신형 투아렉' 국내 상륙[머니S픽]
링크 = https://n.news.naver.com/mnews/article/417/0001019102
----------
news 제목 = 브로드웨이 마법을 서울로… ‘알라딘’ 韓초연 캐스팅도 역대급
링크 = https://n.news.naver.com/mnews/article/023/0003850834
----------
news 제목 = 기록적 폭염에 폭우까지…전국 곳곳서 정전 등 피해 속출
링크 = https://n.news.naver.com/mnews/article/081/0003470285
----------
news 제목 = 두 달 연속 신장세 르노코리아...'그랑 콜레오스'로 부스터 달까
링크 = https://n.news.naver.com/mnews/article/008/0005073110
----------
news 제목 = 국립현대미술관 정영선 전시 디자인, 독일 레드닷 디자인상 받아
링크 = https://n.news.naver.com/mnews/article/055/0001179095
----------
news 제목 = [날씨] 

### 2. Image 다운로드
* referer 요청 헤더를 반드시 설정해야 한다.

In [53]:
import requests
import os

req_header = {
    'referer':'https://comic.naver.com/webtoon/detail?titleId=798173&no=5&amp;weekday=thu'
}
# jpg filename lsit
img_urls = [
    'https://image-comic.pstatic.net/webtoon/798173/5/20220804112251_d97bd1e1b38f0cd022e4e3639d2926b3_IMAG01_1.jpg',
    'https://image-comic.pstatic.net/webtoon/798173/5/20220804112251_d97bd1e1b38f0cd022e4e3639d2926b3_IMAG01_2.jpg',
    'https://image-comic.pstatic.net/webtoon/798173/5/20220804112251_d97bd1e1b38f0cd022e4e3639d2926b3_IMAG01_3.jpg'
]
for img_url in img_urls:
    res = requests.get(img_url, headers=req_header)
    if res.ok:
        # binary 데이터 추출
        img_data = res.content
        # url에서 file name을 가져오기
        file_name = os.path.basename(img_url)
        with open(file_name, 'wb') as file:
            print(f'writing to {file_name} ({len(img_data):,} bytes)')
            file.write(img_data)
    else:
        print(f'Error 발생 {res.status_code}')

writing to 20220804112251_d97bd1e1b38f0cd022e4e3639d2926b3_IMAG01_1.jpg (299,319 bytes)
writing to 20220804112251_d97bd1e1b38f0cd022e4e3639d2926b3_IMAG01_2.jpg (163,333 bytes)
writing to 20220804112251_d97bd1e1b38f0cd022e4e3639d2926b3_IMAG01_3.jpg (182,530 bytes)


* 현재 요청된 페이지(특정 회차)의 image 모두 다운로드 해보기

In [72]:
import requests
import os

webtoon_url = 'https://comic.naver.com/webtoon/detail?titleId=817945&no=37&week=mon'
req_header = {
    'referer': webtoon_url
}
# jpg image 파일명을 먼저 알아내기
res = requests.get(webtoon_url)
if res.ok: #200
    # xxx.jpg 파일명 목록 가져오기
    soup = BeautifulSoup(res.text, 'html.parser')
    print(len(soup.select("img[src$='.jpg']"))) # 108개
    print(len(soup.select("img[src*='IMAG01']"))) # 18개 <- 채택
    img_tag_list = soup.select("img[src*='IMAG01']")
    
    # image 파일명을 저장할 리스트 선언
    img_url_list = []
    for img_tag in img_tag_list:
        '''
            <img src="https://image-comic.pstatic.net/webtoon/817945/37/_IMAG01_1.jpg"/>
        '''
        # print(img_tag['src'])
        # image 태그의 src속성의 값을 추출해서 list에 추가하기
        # ['http://x1.jpg', 'http://x2.jpg']
        img_url_list.append(img_tag['src'])
    
    image_dir_name = 'img'
    # img 디렉토리가 없으면 img 디렉토리 생성하기
    if not os.path.isdir(image_dir_name):
        os.mkdir(image_dir_name)

    for idx, img_url in enumerate(img_url_list, 1):
        # print(idx, img_url)
        res = requests.get(img_url, headers=req_header)
        print(idx, res.status_code)
        if res.ok:
            # binary 데이터 추출
            img_data = res.content
            # url에서 file name을 가져오기
            file_name = os.path.basename(img_url)
            # img/xxx.jpg
            dir_name = os.path.join(image_dir_name, file_name)
            with open(dir_name, 'wb') as file:
                print(f'writing to {dir_name} ({len(img_data):,} bytes)')
                file.write(img_data)
        else:
            print(f'Error 발생 {res.status_code}')
            
else:
    print(res.status_code)

102
18
1 200
writing to img/20240226100634_548bfa8f14799d53d45b5f12df1b289e_IMAG01_1.jpg (128,897 bytes)
2 200
writing to img/20240226100634_548bfa8f14799d53d45b5f12df1b289e_IMAG01_2.jpg (143,194 bytes)
3 200
writing to img/20240226100634_548bfa8f14799d53d45b5f12df1b289e_IMAG01_3.jpg (101,900 bytes)
4 200
writing to img/20240226100634_548bfa8f14799d53d45b5f12df1b289e_IMAG01_4.jpg (112,007 bytes)
5 200
writing to img/20240226100634_548bfa8f14799d53d45b5f12df1b289e_IMAG01_5.jpg (190,876 bytes)
6 200
writing to img/20240226100634_548bfa8f14799d53d45b5f12df1b289e_IMAG01_6.jpg (134,590 bytes)
7 200
writing to img/20240226100634_548bfa8f14799d53d45b5f12df1b289e_IMAG01_7.jpg (132,910 bytes)
8 200
writing to img/20240226100634_548bfa8f14799d53d45b5f12df1b289e_IMAG01_8.jpg (174,434 bytes)
9 200
writing to img/20240226100634_548bfa8f14799d53d45b5f12df1b289e_IMAG01_9.jpg (88,576 bytes)
10 200
writing to img/20240226100634_548bfa8f14799d53d45b5f12df1b289e_IMAG01_10.jpg (177,240 bytes)
11 200
writi

### 3. 파일 업로드 하기
* http://httpbin.org/post 업로드 요청을 할 수 있는 url

### 4. 단축 URL 생성하는 Open API 사용하기
* urllib 사용

In [ ]:
import os
import sys
import urllib.request

client_id = "V" # 개발자센터에서 발급받은 Client ID 값
client_secret = "y_" # 개발자센터에서 발급받은 Client Secret 값
encText = urllib.parse.quote("https://drive.google.com/drive/u/0/folders/1V_DK7Px5w_niEKLlWhDr_GPgIYHG3Du-")
data = "url=" + encText
url = "https://openapi.naver.com/v1/util/shorturl"

request = urllib.request.Request(url)
request.add_header("X-Naver-Client-Id",client_id)
request.add_header("X-Naver-Client-Secret",client_secret)

response = urllib.request.urlopen(request, data=data.encode("utf-8"))
rescode = response.getcode()
if(rescode==200):
    response_body = response.read()
    print(response_body.decode('utf-8'))
else:
    print("Error Code:" + rescode)


* requests를 사용하는 코드로 변경하기
* [requests docs](https://requests.readthedocs.io/en/latest/user/quickstart/)

In [ ]:
import requests

client_id = "V" # 개발자센터에서 발급받은 Client ID 값
client_secret = "y_" # 개발자센터에서 발급받은 Client Secret 값

origin_url = "https://drive.google.com/drive/u/0/folders/1V_DK7Px5w_niEKLlWhDr_GPgIYHG3Du-"



### 5. 블로그 검색하기

In [ ]:
import requests
import pprint

headers = {
    'X-Naver-Client-Id': 'V',
    'X-Naver-Client-Secret': 'y_',
}

payload = {
    'query': '파이썬',
    'display': 100,
    'sort': 'sim'
}

url = 'https://openapi.naver.com/v1/search/blog.json'

res = requests.get(url, params=payload, headers=headers)
items_data = res.json()['items']
#print(items_data)

items_list = list()
item_list = []
for item in items_data:
#     print(item)
    item_list.append(item['title'])
    item_list.append(item['bloggername'])
    item_list.append(item['description'])
    item_list.append(item['bloggerlink'])
    item_list.append(item['link'])

    items_list.append(item_list)
    item_list = []

print(items_list)

with open('data/nhnblog.txt','w',encoding="utf-8")as file:
    for items in items_list:
        for item in items:
            item = item + '\n'
            file.write(item)
        file.write('-'*150+'\n')